In [38]:
import numpy as np # type: ignore # linear algebra
import pandas as pd 

df = pd.read_csv('data_balita.csv')
df


,Umur (bulan),Jenis Kelamin,Tinggi Badan (cm),Status Gizi
0,0,laki-laki,44.591973,stunted
1,0,laki-laki,56.705203,tinggi
2,0,laki-laki,46.863358,normal
3,0,laki-laki,47.508026,normal
4,0,laki-laki,42.743494,severely stunted
...,...,...,...,...
120994,60,perempuan,100.600000,normal
120995,60,perempuan,98.300000,stunted
120996,60,perempuan,121.300000,normal
120997,60,perempuan,112.200000,normal


In [52]:
#Renombrar las columnas
df = df.rename(columns={
    'Umur (bulan)': 'Age (Month)',
    'Jenis Kelamin': 'Gender',
    'Tinggi Badan (cm)': 'Body height',
    'Status Gizi': 'Status'
})
df['Gender'] = df['Gender'].replace({
    'perempuan': '1',
    'laki-laki': '0'
})
df['Status'] = df['Status'].replace({'severely stunted': 0, 'stunted': 1, 'normal': 2, 'tinggi': 3})

In [63]:
# Verificar el cambio
print(df.columns)
print('-----' * 14)
print(df)

Index(['Age (Month)', 'Gender', 'Body height', 'Status'], dtype='object')
----------------------------------------------------------------------
        Age (Month) Gender  Body height  Status
0                 0      0    44.591973       1
1                 0      0    56.705203       3
2                 0      0    46.863358       2
3                 0      0    47.508026       2
4                 0      0    42.743494       0
...             ...    ...          ...     ...
120994           60      1   100.600000       2
120995           60      1    98.300000       1
120996           60      1   121.300000       2
120997           60      1   112.200000       2
120998           60      1   109.800000       2

[120999 rows x 4 columns]


In [66]:
from sklearn.model_selection import train_test_split
'''Se va a entrenar el modelo para que prediga el diagnistico del 
retraso en talla de un bebe menor de 60 meses, pasandole los datos de
edad, genero y altura.'''
# Separar las caracteristicas y el target
X = df.iloc[:, 1:6]
X = df.drop('Status', axis=1)
y = df['Status']

# Dividir el conjunto de datos en datos de entrenamiento y datos de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X

,Age (Month),Gender,Body height
0,0,0,44.591973
1,0,0,56.705203
2,0,0,46.863358
3,0,0,47.508026
4,0,0,42.743494
...,...,...,...
120994,60,1,100.600000
120995,60,1,98.300000
120996,60,1,121.300000
120997,60,1,112.200000


In [44]:
# entrenar el modelo.
from sklearn.neighbors import KNeighborsClassifier

#Crear un objetoo KNN con k=3
knn = KNeighborsClassifier(algorithm='kd_tree', n_neighbors=1)

# Entrenar el modelo con los datos de entrenamiento.
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='kd_tree', n_neighbors=1)

In [45]:
from sklearn.metrics import accuracy_score

# Hacer predicciones con los datos de prueba
y_pred = knn.predict(X_test)

# Calcular la presición del modelo.
accuracy = accuracy_score(y_test, y_pred)
print('Akurasi model KNN:', accuracy)

Akurasi model KNN: 0.9990495867768595


In [46]:
from sklearn.metrics import classification_report, confusion_matrix

print('confusion_matrix knn:')
confusion_matrix(y_test, y_pred)

confusion_matrix knn:


array([[ 4129,     1,     0,     0],
       [    6,  2782,     2,     0],
       [    0,     3, 13372,     7],
       [    0,     0,     4,  3894]], dtype=int64)

In [47]:
print(classification_report(y_test.values.reshape(-1, 1), y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4130
           1       1.00      1.00      1.00      2790
           2       1.00      1.00      1.00     13382
           3       1.00      1.00      1.00      3898

    accuracy                           1.00     24200
   macro avg       1.00      1.00      1.00     24200
weighted avg       1.00      1.00      1.00     24200



In [48]:
from sklearn.model_selection import cross_val_score

print('Cross val: ', cross_val_score(knn, y_test.values.reshape(-1, 1), y_pred, cv = 5))
print('Accuracy : ', np.mean(cross_val_score(knn, y_test.values.reshape(-1, 1), y_pred)))

Cross val:  [0.99855372 0.99876033 0.99917355 0.99958678 0.99917355]
Accuracy :  0.9990495867768594


In [76]:
def interpretation(pred):
    if pred == 0:
        return "Severely stuned"
    elif pred == 1: 
        return 'Stuned'
    elif pred == 2: 
        return 'Normal'
    else: 
        return 'Tall'

test = pd.DataFrame({'Age (Month)': [16], 'Gender': [1], 'Body height': [70]})
pred = knn.predict(test)

interpretation(pred)

'Severely stuned'